# WHISP pure Cloud Function

In [ ]:
REGION = 'us-central1'
PROJECT = 'forest-data-partnership'

In [ ]:
!gcloud auth login --project {PROJECT} --billing-project {PROJECT} --update-adc

## Create the Cloud function and deploy it

In [ ]:
!mkdir whisper

Get the list of datasets from the WHISP GitHub repo.

In [ ]:
!curl https://raw.githubusercontent.com/forestdatapartnership/whisp/main/src/openforis_whisp/datasets.py --output whisper/datasets.py

# EEasify WHISP

In [ ]:
%%writefile whisper/easy_whisp.py

import google.auth
import ee

# First, initialize.
credentials, _ = google.auth.default(
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials, project='forest-data-partnership', opt_url='https://earthengine-highvolume.googleapis.com')

from datasets import combine_datasets

def easy_whisp() -> ee.Image:
    """Returns the stack."""
    return combine_datasets()

In [ ]:
%%writefile whisper/main.py

import json
import ee
from flask import jsonify
import functions_framework
import logging
import requests

import google.auth
import google.cloud.logging
from google.api_core import retry

from easy_whisp import easy_whisp

client = google.cloud.logging.Client()
client.setup_logging()


@retry.Retry()
def get_whisp_stats(geojson):
    """"""
    region = ee.Geometry(geojson)
    whisp_image = easy_whisp()
    return whisp_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=10
    ).getInfo()


@functions_framework.http
def main(request):
  """"""
  credentials, _ = google.auth.default(
      scopes=['https://www.googleapis.com/auth/earthengine']
  )
  ee.Initialize(credentials, project='forest-data-partnership')
  try:
    replies = []
    request_json = request.get_json(silent=True)
    calls = request_json['calls']
    for call in calls:
      geo_json = json.loads(call[0])
      try:
        logging.info([geo_json])
        response = get_whisp_stats(geo_json)
        logging.info(response)
        replies.append(json.dumps(response))
      except Exception as e:
        logging.error(str(e))
        replies.append(json.dumps( { "errorMessage": str(e) } ))
    return jsonify(replies=replies, status=200, mimetype='application/json')
  except Exception as e:
    error_string = str(e)
    logging.error(error_string)
    return jsonify(error=error_string, status=400, mimetype='application/json')

In [ ]:
%%writefile whisper/requirements.txt
earthengine-api
flask
functions-framework
google-api-core
google-cloud-logging
requests

In [ ]:
!gcloud functions deploy 'whisper' \
  --gen2 \
  --region={REGION} \
  --project={PROJECT} \
  --runtime=python312 \
  --source='whisper' \
  --entry-point=main \
  --trigger-http \
  --no-allow-unauthenticated \
  --timeout=120s

## Load WHISP example data

Here we will get the WHISP example data from GitHub, convert to line delimited GeoJSON...

In [ ]:
import json

In [ ]:
fc_list = !curl https://raw.githubusercontent.com/forestdatapartnership/whisp/main/tests/fixtures/geojson_example.geojson

In [ ]:
fc_obj = json.loads("\n".join(fc_list))

In [ ]:
features = fc_obj['features']

In [ ]:
feature = features[0]

In [ ]:
feature

In [ ]:
geoms = [f['geometry'] for f in features]

In [ ]:
geoms[0]

In [ ]:
json.dumps(geoms[0], separators=(',', ':'))

In [ ]:
import ee
ee.Initialize(project='forest-data-partnership')

In [ ]:
print(ee.Geometry(geoms[0]).getInfo())

## Test the deployed Cloud Function

In [ ]:
!gcloud auth print-identity-token

In [ ]:
import json

test_calls = [[json.dumps(g), '2021-01-01', '2021-12-31'] for g in geoms]
test_request = json.dumps({'calls': test_calls}, separators=(',', ':')).join("''")

In [ ]:
test_request

In [ ]:
!curl -X POST https://{REGION}-{PROJECT}.cloudfunctions.net/whisper \
  -H "Authorization: bearer $(gcloud auth print-identity-token)" \
  -H "Content-Type: application/json" \
  -d {test_request}